# langChain Agent

1.initialize_agent,：执行gent工作，并把工具Tool传入

2.Tool：选取行为函数工具类

之前我们学习的都是把问题给AI，让AI模型给出答案，那么这种情况下应该怎么处理呢，我需要根据不同的问题选择不同的答案，比如我问AI我想选择一件衣服就去调用挑选衣服的方法，如果是查询订单，那么就专门去调用搜索订单的方法，如果是查询物流就专门去调用物流方面的方法，但是怎么识别出来哪个调用哪个呢？

In [41]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [42]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
 
llm=ChatOpenAI(max_tokens=2048,temperature=0.5)
multiple_choice="""
请针对 >>> 和 <<< 中间的用户问题，选择一个合适的工具去回答它的问题。只要用A、B、C的选项字母告诉我答案。
如果你觉得都不合适，就选D。
>>>{question}<<<
我们有的工具包括：
A. 一个能够查询商品信息，为用户进行商品导购的工具
B. 一个能够查询订单信息，获得最新的订单情况的工具
C. 一个能够搜索商家的退换货政策、运费、物流时长、支付渠道、覆盖国家的工具
D. 都不合适
"""
multiple_choice_prompt=PromptTemplate(template=multiple_choice,input_variables=["question"])
choice_chain=LLMChain(llm=llm,prompt=multiple_choice_prompt, output_key="answer")


In [43]:
question = "我想买一件衣服，但是不知道哪个款式好看，你能帮我推荐一下吗？"
print(choice_chain(question))

{'question': '我想买一件衣服，但是不知道哪个款式好看，你能帮我推荐一下吗？', 'answer': 'A. 一个能够查询商品信息，为用户进行商品导购的工具。'}


In [44]:
question = "我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？"
print(choice_chain(question))

{'question': '我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？', 'answer': 'B. 一个能够查询订单信息，获得最新的订单情况的工具。'}


In [45]:
question = "请问你们的货，能送到三亚吗？大概需要几天？"
print(choice_chain(question))

{'question': '请问你们的货，能送到三亚吗？大概需要几天？', 'answer': 'C. 一个能够搜索商家的退换货政策、运费、物流时长、支付渠道、覆盖国家的工具\n\n根据用户的问题，需要查询商家的运费、物流时长等信息，因此选择C选项的工具最合适。'}


In [46]:
question = "今天天气怎么样？"
print(choice_chain(question))

{'question': '今天天气怎么样？', 'answer': 'D. 都不合适\n\n以上提到的工具都不适合回答关于天气的问题。可以使用天气预报网站或者手机应用来获取今天的天气情况。'}


Agent在这里既有中介也有特工的意思，它能根据你提供的要求既能选择出工具并还有执行能力，

代码如下：

我们定义了三个函数

1、search_order()第一个函数是关于订单搜索，最后模拟时返回订单信息

2、recommend_product()定义了关于推荐产品的函数

3、faq()定义了电商问答的函数

然后我们最重要的就是tools数组，数组里放入tool函数，参数为name，func是函数，不同的功能调用不同的函数，description这个主要是类似于Prompt，根据用户输入的是否符合描述来调用不同的函数。


In [47]:
from langchain.agents import initialize_agent, Tool, create_react_agent
from langchain_openai import ChatOpenAI
 
llm=ChatOpenAI(temperature=0)
 
# 模拟问关于订单
def search_order(input:str) ->str:
  return "订单状态：已发货；发货日期：2023-09-15；预计送达时间：2023-09-18"
 
# 模拟问关于推荐产品
def recommend_product(input:str)->str:
  return "红色连衣裙"
 
# 模拟问电商faq
def faq(input:str)->str:
  return "7天无理由退货"
 
# 创建了一个 Tool 对象的数组，把这三个函数分别封装在了三个 Tool 对象里面
# 并且定义了描述，这个 description 就是告诉 AI，这个 Tool 是干什么用的，会根据描述做出选择
tools=[
    Tool(
        name="Search Order",func=search_order,
        description="useful for when you need to answer questions about customers orders"
    ),
    Tool(
        name="Recommend Product",func=recommend_product,
        description="useful for when you need to answer questions about product recommendations"
    ),
    Tool(
        name="FAQ",func=faq,
        description="useful for when you need to answer questions about shopping policies, like return policy, shipping policy, etc."
    ),
]
# 指定使用tools，llm，agent则是zero-shot"零样本分类"，不给案例自己推理
# 而 react description，指的是根据你对于 Tool 的描述（description）进行推理（Reasoning）并采取行动（Action）
agent=initialize_agent(tools,llm,agent="zero-shot-react-description", verbose=True)

# template = '''尽可能回答以下问题。您可以使用以下工具：

# {tools}

# 使用以下格式

# Question: 您必须回答的输入问题
# Thought: 你应该时刻思考该做什么
# Action: the action to take, should be one of [{tool_names}]
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question

# Begin!

# Question: {input}
# Thought:{agent_scratchpad}'''

# prompt = PromptTemplate.from_template(template)
# agent = create_react_agent(llm, tools, prompt=prompt)

In [48]:
question = "我想买一件衣服，但是不知道哪个款式好看，你能帮我推荐一下吗？"
result=agent.run(question)
print(result)



> Entering new AgentExecutor chain...
I should recommend a product based on the customer's preferences.
Action: Recommend Product
Action Input: Clothing category
Observation: 红色连衣裙
Thought:I should provide more options for the customer to choose from.
Action: Recommend Product
Action Input: Dresses category
Observation: 红色连衣裙
Thought:The customer seems to be interested in a red dress, so I should recommend more red dresses.
Action: Recommend Product
Action Input: Red dresses category
Observation: 红色连衣裙
Thought:I now know the final answer
Final Answer: 我推荐您购买一条红色连衣裙。

> Finished chain.
我推荐您购买一条红色连衣裙。


首先讲讲它打印的日志：

Action：就是根据用户的输入，选用哪个Tool，然后行动

Action Input：根据需要使用的Tool，从用户的输入里提取相关的内容，可以输入到Tool里面

Observation：就是观察通过使用 Tool 得到的一个输出结果。

Thought：就是再看一眼用户的输入，判断一下该怎么做。

Final Answer：就是 Thought 在看到 Obersavation 之后，给出的最终输出。


In [49]:
question = "我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
I need to search for the order with the order number provided.
Action: Search Order
Action Input: 2022ABCDE
Observation: 订单状态：已发货；发货日期：2023-09-15；预计送达时间：2023-09-18
Thought:The order has been shipped and is expected to arrive on September 18, 2023.
Final Answer: Your order with order number 2022ABCDE has been shipped and is expected to arrive on September 18, 2023.

> Finished chain.
Your order with order number 2022ABCDE has been shipped and is expected to arrive on September 18, 2023.


In [50]:
question = "请问你们的货，能送到三亚吗？大概需要几天？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
I need to find out about the shipping policy and estimated delivery time to Sanya.
Action: FAQ
Action Input: Shipping policy
Observation: 7天无理由退货
Thought:I need more specific information about delivery time to Sanya.
Action: FAQ
Action Input: Delivery time to Sanya
Observation: 7天无理由退货
Thought:I need to provide the customer with more accurate information about delivery time to Sanya.
Action: FAQ
Action Input: Estimated delivery time to Sanya
Observation: 7天无理由退货
Thought:I need to inform the customer that the estimated delivery time to Sanya is typically 7 days.
Final Answer: 我们的货物通常可以在7天内送达三亚。

> Finished chain.
我们的货物通常可以在7天内送达三亚。


## 使用max_iterations限制重试次数
我们可以把上面的案例设置为2，看看代码

In [51]:
agent=initialize_agent(tools,llm,agent="zero-shot-react-description",max_iterations=2,verbose=True)
question = "请问你们的货，能送到三亚吗？大概需要几天？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
I need to find out about the shipping policy and estimated delivery time to Sanya.
Action: FAQ
Action Input: Shipping policy to Sanya
Observation: 7天无理由退货
Thought:I need more specific information about the estimated delivery time to Sanya.
Action: FAQ
Action Input: Estimated delivery time to Sanya
Observation: 7天无理由退货
Thought:

> Finished chain.
Agent stopped due to iteration limit or time limit.


## 通过 VectorDBQA 让 Tool 支持问答
当然对于上面那种回答不了的问题，我们也可以采用之前说过的方法，我们调用faq对话，将VectorDBQA 这个 LLMChain，把它也封装成一个 Tool
也就是说我们将FAQ对话文件读取，读取完按一小块存储，并最后存储成向量，我们根据这个处理完的数据查询

In [52]:
#!pip install spacy;
#!python -m spacy download zh_core_web_sm
# !pip install tiktoken
#!pip install faiss-cpu
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import SpacyTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.agents import initialize_agent
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.chains import VectorDBQA 
 
llm = ChatOpenAI(temperature=0)
loader = TextLoader('./data/wonderland.txt')
documents = loader.load()
text_splitter = SpacyTextSplitter(chunk_size=256, pipeline="zh_core_web_sm")
texts = text_splitter.split_documents(documents)
 
embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_documents(texts, embeddings)
 
faq_chain = VectorDBQA.from_chain_type(llm=llm, vectorstore=docsearch, verbose=True)

/slurm/home/yrd/shaolab/daiyizheng/.conda/envs/langechain/lib/python3.10/site-packages/langchain/chains/retrieval_qa/base.py:256: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


然后，把这 LLMChain 的 run 方法包装到一个 Tool 里面。也就是关于faq直接就在方法里调用faq_chain.run去调用我们文件的数据了。

In [53]:
# Python 的 decorator 功能，相当于用了tool的函数功能
@tool("FAQ")
def faq(input:str) -> str:
  """"useful for when you need to answer questions about shopping policies, like return policy, shipping policy, etc."""
  return faq_chain.run(input)
 
tools=[
    Tool(
        name="Search Order",func=search_order,
        description="useful for when you need to answer questions about customers orders"
    ),
    Tool(
        name="Recommend Product",func=recommend_product,
        description="useful for when you need to answer questions about product recommendations"
    ),
    faq
]
agent=initialize_agent(tools,llm,agent="zero-shot-react-description",verbose=True)

In [54]:
from pprint import pprint
pprint(agent.agent.llm_chain.prompt.template)

('Answer the following questions as best you can. You have access to the '
 'following tools:\n'
 '\n'
 'Search Order: useful for when you need to answer questions about customers '
 'orders\n'
 'Recommend Product: useful for when you need to answer questions about '
 'product recommendations\n'
 'FAQ: FAQ(input: str) -> str - "useful for when you need to answer questions '
 'about shopping policies, like return policy, shipping policy, etc.\n'
 '\n'
 'Use the following format:\n'
 '\n'
 'Question: the input question you must answer\n'
 'Thought: you should always think about what to do\n'
 'Action: the action to take, should be one of [Search Order, Recommend '
 'Product, FAQ]\n'
 'Action Input: the input to the action\n'
 'Observation: the result of the action\n'
 '... (this Thought/Action/Action Input/Observation can repeat N times)\n'
 'Thought: I now know the final answer\n'
 'Final Answer: the final answer to the original input question\n'
 '\n'
 'Begin!\n'
 '\n'
 'Question: {inp

In [55]:
question = "宇智波鼬？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
I don't recognize this as a question related to customer orders, product recommendations, or shopping policies.
Action: None
Final Answer: I'm sorry, I cannot provide an answer to that question.

> Finished chain.
I'm sorry, I cannot provide an answer to that question.


对于商品的推荐，我们也可以使用类似方式，也把对应的商品信息，存到 VectorStore 里，然后通过先搜索后问答的方式来解决。




In [56]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import CSVLoader
 
product_loader = CSVLoader('./data/ecommerce_products.csv')
product_documents = product_loader.load()
product_text_splitter = CharacterTextSplitter(chunk_size=1024, separator="\n")
product_texts = product_text_splitter.split_documents(product_documents)
product_search = FAISS.from_documents(product_texts, OpenAIEmbeddings())
product_chain = VectorDBQA.from_chain_type(llm=llm, vectorstore=product_search, verbose=True)
 
@tool("FAQ")
def faq(intput: str) -> str:
    """"useful for when you need to answer questions about shopping policies, like return policy, shipping policy, etc."""
    return faq_chain.run(intput)
 
@tool("Recommend Product")
def recommend_product(input:str) ->str:
  """"useful for when you need to search and recommend products and recommend it to the user"""
  return product_chain.run(input)
 
tools=[
    recommend_product,
    faq
]
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

question = "我想买一件衣服，想要在春天去公园穿，但是不知道哪个款式好看，你能帮我推荐一下吗？"
answer = agent.run(question)
print(answer)

/slurm/home/yrd/shaolab/daiyizheng/.conda/envs/langechain/lib/python3.10/site-packages/langchain/chains/retrieval_qa/base.py:256: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(




> Entering new AgentExecutor chain...
I need to recommend a clothing item for the user to wear in the spring to the park.
Action: Recommend Product
Action Input: "spring park outfit"

> Entering new VectorDBQA chain...

> Finished chain.

Observation: I don't know.
Thought:I need to provide a more specific input to get a better recommendation.
Action: Recommend Product
Action Input: "spring park outfit women"

> Entering new VectorDBQA chain...

> Finished chain.

Observation: For a spring park outfit for women, you can consider the following options based on the styles mentioned in the product descriptions:

1. **长袖连衣裙 (Long Sleeve Dress)**: A soft and comfortable pink long sleeve dress, with a fresh and sweet style. Pair it with white sneakers and a small bag for a natural and fresh look.

2. **长款风衣 (Long Windbreaker)**: A warm khaki windbreaker with a casual and simple design. Ideal for spring weather. Combine it with white sneakers and jeans for a relaxed and natural style.

3. *

## 优化 Prompt，让 AI 不要胡思乱想

In [57]:
import json
import re
 
ORDER_1="20230101ABC"
ORDER_2="20230101EFG"
 
ORDER_1_DETAIL={
    "order_number":ORDER_1,
    "status":"已发货",
    "shipping_date": "2023-01-03",    
    "estimated_delivered_date": "2023-01-05",
}
 
ORDER_2_DETAIL={
    "order_number":ORDER_2,
    "status":"未发货",
    "shipping_date": None,    
    "estimated_delivered_date": None,
}
 
@tool("SearchOrder")
def search_order(input:str)->str:
  """useful for when you need to answer questions about customers orders"""
  if input.strip()==ORDER_1:
    return json.dumps(ORDER_1_DETAIL)
  elif input.strip()==ORDER_2:
    return json.dumps(ORDER_2_DETAIL)
  else:
    return f"对不起，根据{input}没有找到您的订单"
tools=[search_order, recommend_product, faq]
agent=initialize_agent(tools, llm=ChatOpenAI(temperature=0), agent="zero-shot-react-description", verbose=True)

在这里我们匹配到的就直接返回了和我们期望一致的数据，那如果我们查询一下上面没有的订单号，也就是匹配不上，会出现什么

In [59]:
question = "我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？"
answer = agent.run(question)
print(answer)



> Entering new AgentExecutor chain...
I need to help the customer track their order.
Action: SearchOrder
Action Input: 2022ABCDE
Observation: 对不起，根据2022ABCDE没有找到您的订单
Thought:The order number provided does not exist in the system.
Action: SearchOrder
Action Input: N/A
Observation: 对不起，根据N/A没有找到您的订单
Thought:The customer may have provided the wrong order number.
Action: FAQ
Action Input: "如何处理找不到订单的情况？"

> Entering new VectorDBQA chain...

> Finished chain.

Observation: 抱歉，根据提供的文本内容，我无法回答关于订单的问题。如果您有其他问题，我会尽力帮助您。
Thought:I need to provide the customer with a way to contact customer service for further assistance.
Action: FAQ
Action Input: "如何联系客服？"

> Entering new VectorDBQA chain...

> Finished chain.

Observation: 抱歉，根据提供的文本，我无法确定如何联系客服。
Thought:The customer may need to contact customer service directly for help with their order.
Final Answer: I'm sorry, but I am unable to track your order. I recommend contacting customer service for further assistance.

> Finished chain.
I'm sorry, 

最终返回的结果不是我们要的结果，想要让它返回对不起，根据2022ABCDE没有找到您的订单，但他最后却重复调用 OpenAI 的思考策略，并最终尝试从 FAQ 里拿一个查询订单的问题来敷衍用户。

解决的方案就是更改下提示语，通过这个提示语，Agent 会知道，这个工具就应该在找不到订单的时候，告诉用户找不到订单或者请它再次确认。


In [60]:
@tool("SearchOrder")
def search_order(input:str)->str:
  """一个帮助用户查询最新订单状态的工具，并且能处理以下情况：    
  1. 在用户没有输入订单号的时候，会询问用户订单号    
  2. 在用户输入的订单号查询不到的时候，会让用户二次确认订单号是否正确"""
  
  # 匹配多个数字以及大写字母
  pattern=r"\d+[A-Z]+"
  match=re.search(pattern,input)
  order_number=input
  if match:
    # 得到整个订单字符串
    order_number=match.group(0)
  else:
    return "请问您的订单号是多少？"
 
  if order_number==ORDER_1:
    return json.dumps(ORDER_1_DETAIL)
  elif order_number==ORDER_2:
    return json.dumps(ORDER_2_DETAIL)
  else:
    return f"对不起，根据{input}没有找到您的订单"
tools=[search_order,recommend_product,faq]
agent=initialize_agent(tools,llm=ChatOpenAI(temperature=0),agent="zero-shot-react-description", verbose=True)
 
question = "我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？"
answer = agent.run(question)
print(answer)



> Entering new AgentExecutor chain...
这是一个关于订单状态的问题，需要使用SearchOrder工具来查询订单状态。
Action: SearchOrder
Action Input: 2022ABCDE
Observation: 对不起，根据2022ABCDE没有找到您的订单
Thought:订单号可能输入错误了，需要再次确认一下。
Action: SearchOrder
Action Input: 2022ABCDE
Observation: 对不起，根据2022ABCDE没有找到您的订单
Thought:用户可能需要再次确认订单号是否正确，可以询问用户是否输入正确的订单号。
Action: SearchOrder
Action Input: 2022ABCDE
Observation: 对不起，根据2022ABCDE没有找到您的订单
Thought:用户可能需要帮助确认订单号是否正确，可以建议用户再次检查订单号并提供帮助。
Action: Recommend Product
Action Input: 提醒用户再次检查订单号是否正确

> Entering new VectorDBQA chain...

> Finished chain.

Observation: 抱歉，我不知道。
Thought:用户可能需要了解更多关于订单状态的信息，可以提供FAQ来回答关于订单状态的常见问题。
Action: FAQ
Action Input: 订单状态

> Entering new VectorDBQA chain...

> Finished chain.

Observation: 抱歉，我无法回答关于订单状态的问题。
Thought:用户可能需要了解更多关于订单状态的信息，可以提供FAQ来回答关于订单状态的常见问题。
Action: FAQ
Action Input: 订单状态

> Entering new VectorDBQA chain...

> Finished chain.

Observation: 抱歉，我无法回答关于订单状态的问题。
Thought:经过多次尝试，无法通过SearchOrder工具查询到订单状态，建议用户直接联系客服部门查询订单状态。
Final Answer: 建议用户直接联系客服

## 订单查询多伦对话
我们再继续优化：

我们应该支持多轮聊天。因为用户不一定是在第一轮提问的时候，就给出了自己的订单号。

我们其实可以直接让 Search Order 这个 Tool，回答用户的问题，没有必要再让 Agent 思考一遍，使用return_direct这个参数就不回在经过Thought，直接把回答传给用户。


In [61]:
from langchain.memory import ConversationBufferMemory
from langchain_openai  import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
 
answer_order_info=PromptTemplate(
    template="请把下面的订单信息回复给用户： \n\n {order}?", 
    input_variables=["order"]
)
answer_order_llm=LLMChain(llm=ChatOpenAI(temperature=0), prompt=answer_order_info)
 
# return_direct这个参数是不要再经过Thought，直接把回答传给用户
@tool("SearchOrder",return_direct=True)
def search_order(input:str)->str:
  """useful for when you need to answer questions about customers orders"""
  pattern=r"\d+[A-Z]+"
  match=re.search(pattern,input)
  order_number=input
  if match:
    # 取出整个订单字符集
    order_number=match.group(0)
  else:
    return "请问您的订单号是多少？"
 
  if order_number==ORDER_1:
    # 通过 answer_order_llm 这个工具来组织语言文字，将python对象编码成Json字符串
    return answer_order_llm.run(json.dumps(ORDER_1_DETAIL)) 
  elif order_number==ORDER_2:
    return answer_order_llm.run(json.dumps(ORDER_2_DETAIL))
  else:
    return f"对不起，根据{input}没有找到您的订单"
 
tools=[search_order,recommend_product,faq]
# 更换为ChatOpenAI成本更低
chatllm=ChatOpenAI(temperature=0)
# 保留对话记录到内存
memory=ConversationBufferMemory(memory_key="chat_history",return_message=True)
# 更改agent为conversational-react-description支持多语言对话
conversation_agent=initialize_agent(tools,chatllm,
                                    agent="conversational-react-description",
                                    memory=memory, 
                                    verbose=True)

In [62]:
question1 = "我有一张订单，一直没有收到，能麻烦帮我查一下吗？"
answer1 = conversation_agent.run(question1)
print(answer1)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: SearchOrder
Action Input: 我有一张订单，一直没有收到
Observation: 请问您的订单号是多少？


> Finished chain.
请问您的订单号是多少？


In [63]:
question2 = "我的订单号是20230101ABC"
answer2 = conversation_agent.run(question2)
print(answer2)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: SearchOrder
Action Input: 20230101ABC
Observation: 您的订单信息如下：
订单号：20230101ABC
状态：已发货
发货日期：2023-01-03
预计送达日期：2023-01-05

如有任何问题，请随时与我们联系。感谢您的购买！


> Finished chain.
您的订单信息如下：
订单号：20230101ABC
状态：已发货
发货日期：2023-01-03
预计送达日期：2023-01-05

如有任何问题，请随时与我们联系。感谢您的购买！
